[View in Colaboratory](https://colab.research.google.com/github/Curiousss/FLD/blob/experiments/FLDNetwork.ipynb)

In [1]:
!wget https://www.dropbox.com/s/9j2om0gcv43hkz8/300w_cropped.zip


Redirecting output to ‘wget-log’.


In [0]:
!unzip 300w_cropped.zip

In [3]:
#https://github.com/Curiousss/FLD/blob/master/kernel45.txt
#Get this file
from google.colab import files
uploaded = files.upload()

Saving kernel45.txt to kernel45.txt


In [4]:
!git clone https://github.com/albanie/pts_loader
!ls pts_loader/
!cp pts_loader/pts_loader.py .
!ls

Cloning into 'pts_loader'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), done.
LICENSE.md  pts_loader.py  README.md  tests
01_Indoor   300w_cropped.zip  pts_loader     sample_data
02_Outdoor  kernel45.txt      pts_loader.py  wget-log


In [5]:
import os
from os import listdir,makedirs
from os.path import isfile,join
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import cv2

import tensorflow as tf
import keras.backend as K
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Add, SeparableConv2D
from keras.layers import Convolution2D, Lambda, Input, Reshape
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.layers import LeakyReLU
from keras import optimizers




Using TensorFlow backend.


In [0]:
'''
Hyper-Parameters
'''
image_H = 96.0
image_W = 96.0
grayscale = True
batch_size = 16
learning_rate=0.001
momentum = 0.7
decay = 0
epochs = 100
droupout= 0.2
gradient_clipping = 0.1
nFilters = 128
nlandmarks = nLabels = 68
K=45
img_rows = 96
img_cols = 96
img_channels = 1
alpha = 0.1
epsilon = 0.0001

In [0]:
# Get dataset 
import pts_loader

img_x = []
pts_y = []
for i in range(1,600):
    #print("iteration", i)
    if i == 78 or i == 79 or i == 266 or i == 385 or i == 478 or i == 549 or i == 550:
      continue
    if i < 10:
        img_f = r"01_Indoor/indoor_00" + str(i) +".png"
        pts_f = r"01_Indoor/indoor_00" + str(i) +".pts"
    elif i < 100:
        img_f = r"01_Indoor/indoor_0" + str(i) +".png"
        pts_f = r"01_Indoor/indoor_0" + str(i) +".pts"
    elif i < 301:
        img_f = r"01_Indoor/indoor_" + str(i) +".png"
        pts_f = r"01_Indoor/indoor_" + str(i) +".pts"
    
    elif i < 310:
        j = i - 300
        img_f = r"02_Outdoor/outdoor_00" + str(j) +".png"
        pts_f = r"02_Outdoor/outdoor_00" + str(j) +".pts"
    elif i < 400:
        j = i - 300
        img_f = r"02_Outdoor/outdoor_0" + str(j) +".png"
        pts_f = r"02_Outdoor/outdoor_0" + str(j) +".pts"
    else:
        j = i - 300    
        img_f = r"02_Outdoor/outdoor_" + str(j) +".png"
        pts_f = r"02_Outdoor/outdoor_" + str(j) +".pts"
    

    pts = pts_loader.load(pts_f)
    # Loading imgae in grayscale
    img = cv2.imread(img_f)
    
    # Get the resize scale to adjust the points in landmark images
    scale_x = image_W/img.shape[0]
    scale_y = image_H/img.shape[1]
    img = cv2.resize(img, (img_rows, img_cols))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.normalize(img,None,-1,1,cv2.NORM_MINMAX)
    img = img.reshape( 1, img_rows, img_cols)
    landmarks = []
    landmarks1 = []
    
    #Create 68 blank images with white marks for each of 68 landmarks
    for p in pts:
        new_pts = []
        landmark = np.zeros((96, 96), np.uint8)
        new_pts = (p[0] * scale_y , p[1] * scale_x )
        #new_points.append(new_pts)
        x = int(new_pts[0])
        y = int(new_pts[1])
        #print(n, x, y)
        landmark[x][y] = 1 # Normalized WHite
        landmarks.append(landmark)
        #landmarks1.append(cv2.flip(landmark,1)) # Augmentation: Horizontal flip
        
    #img1 = cv2.flip(img,1) # Augementation: Horizontal flip
    #img1 = img1.reshape(1, img_rows, img_cols)
    img_x.append(img)
    pts_y.append(landmarks)
    #img_x.append(img1)
    #pts_y.append(landmarks1)


In [0]:
#print(pts_y[0])
#assert not np.any(np.isnan(img_x))
#assert not np.any(np.isnan(pts_y))
train_x = []
train_y = []
test_x = []
test_y = []


In [0]:
train_x = np.array(img_x[:500])
train_y = np.array(pts_y[:500])
test_x = np.array(img_x[500:])
test_y = np.array(pts_y[500:])


In [10]:
#train_y = train_y.reshape(-1, 68, 96,96)
#test_y = test_y.reshape(-1, 68, 96,96,68)
print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

(500, 1, 96, 96)
(92, 1, 96, 96)
(500, 68, 96, 96)
(92, 68, 96, 96)


In [0]:
def apply_kernel(local_out):
    '''
     get kernel from text file
     tile it to 1x1x45x45
     convolve with local out
    '''
    f = open(r'kernel45.txt')
    kernel45 = np.fromfile(f, dtype = np.float32, count = -1, sep = " ")
    
    # [1 x 1 x 45 x 45]
    kernel45 = np.reshape(kernel45, (45, 45, 1, 1))
    # [numLabels x 1 x 45 x 45] if tensor convolution is used
    kernel_stacked 	= np.tile(kernel45, (1, 1, nLabels, 1))
    kernel_tensor = tf.convert_to_tensor(kernel_stacked, dtype=tf.float32)
    print("local_out", local_out)
    print("kernel", kernel_tensor)

    return tf.nn.depthwise_conv2d(local_out, kernel_tensor, strides=[1,1,1,1],
                                   padding="SAME", data_format="NCHW")


In [0]:
def FLD_model_build():
  '''
  ----------------------------------------LOCAL SUBNET----------------------------
  '''

  # [ImageC x ImageH x ImageW]
  #-----------------------------------------------5 layers of kernel 5x5----------
  input_img = Input((img_channels, img_rows, img_cols))
  model = SeparableConv2D(filters=nFilters, kernel_size=(5, 5), padding='same', data_format='channels_first')(input_img)
  print("Input model", model)
  model = BatchNormalization(epsilon=epsilon)(model)
  model = LeakyReLU(alpha)(model)
  model = Dropout(droupout)(model)

  for i in range(4):
    model = SeparableConv2D(filters=nFilters, kernel_size=(5, 5), padding='same', data_format='channels_first')(model)
    model = BatchNormalization(epsilon=epsilon)(model)
    model = LeakyReLU(alpha)(model)
    model = Dropout(droupout)(model)

  #-----------------------------------------------10 layers of kernel 3x3----------
  #1
  for i in range(10):
    model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', data_format='channels_first')(model)
    model = BatchNormalization(epsilon=epsilon)(model)
    model = LeakyReLU(alpha)(model)
    model = Dropout(droupout)(model)

  print("Local subnet", model)

  '''
  ----------------------1x1 Convolution with activation-----------------------------
  Apply Gaussian Kernel filter
  '''
  model_Conv = Convolution2D(filters=nLabels, kernel_size=(1, 1), data_format='channels_first')(model)
  model_Conv = LeakyReLU(alpha)(model_Conv)
  model_Conv = Lambda(apply_kernel, output_shape=(nLabels, img_cols, img_rows))(model_Conv)
  print("Main 1x1", model_Conv)
  '''
  -------------------------------------Ol 1x1 Linear convolution-------------------
  Apply Gaussian Kernel filter
  '''
  model_Ol = Convolution2D(filters=nLabels, kernel_size=(1, 1),activation=None, data_format='channels_first')(model)
  model_Ol = Lambda(apply_kernel, output_shape=(nLabels, img_cols, img_rows), name='Ol')(model_Ol)
  print("Local output", model_Ol)

  comb_Ol = keras.layers.concatenate([model_Conv, model_Ol], axis = 1)
  print("Concatenate", comb_Ol)

  '''
  ----------------------------------------GLOBAL SUBNET----------------------------
  '''
  #--------------------------------7 layers of dialated kernel 3x3------------------
  #1
  model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, data_format='channels_first')(comb_Ol)
  model = BatchNormalization(epsilon=epsilon)(model)
  model = LeakyReLU(alpha)(model)
  model = Dropout(droupout)(model)
  #2
  for i in range(6):
    model = SeparableConv2D(filters=nFilters, kernel_size=(3, 3), padding='same', dilation_rate = 4, data_format='channels_first')(model)
    model = BatchNormalization(epsilon=epsilon)(model)
    model = LeakyReLU(alpha)(model)
    model = Dropout(droupout)(model)

  model = Convolution2D(filters=nLabels, kernel_size=(1, 1), activation=None, data_format='channels_first')(model)

  FLD_model = Model(inputs = input_img, outputs=model)
  #FLD_model.summary()
  return FLD_model, model_Ol


In [0]:
def landmark_weight(labels):
    
    #[numLabels x 1 x 1]
    W1 = tf.reduce_sum(labels, axis=1)
    W2 = tf.reduce_sum(W1, axis=0)
    W3 = tf.reduce_mean(W2)
    Win = tf.multiply(W3, 0.5)
    #Win = tf.add(Win, W2)
    #Win = tf.reduce_sum(Win)
    print("Win", Win.shape)
    #Win = tf.reshape(Win, (-1, 1, 1, nLabels))
    return Win

'''
Ol - [numLabels x ImageH x ImageW] Kernel conv already applied in Network
Og - [numLabels x ImageH x ImageW]
labels - Landmark coordinates [numLabels x ImageH x ImageW] without
kernel applied
'''
def loss_function(Ol):
  def FLD_loss_func(y_true, y_pred):
      labels = y_true
      Og = y_pred

      # Ground Truth Heatmap
      Gn = apply_kernel(labels)
      
      Win = landmark_weight(labels)
      global_diff = tf.subtract(Og, Gn)
      global_err = tf.square(global_diff)

      # (Ol - Gn)2

      # Kernel is already applied for Ol
      local_diff = tf.subtract(Ol, Gn)
      local_err = tf.square(local_diff)
      
      comb_err = tf.add(local_err, global_err)
      
      comb_Win = tf.multiply(Win, comb_err)
      wt_square_err = tf.reduce_sum(comb_Win)
      wt_square_err = tf.multiply(wt_square_err, 0.05)
      print("Loss", wt_square_err)
      return wt_square_err    
      
      #return tf.reduce_sum(comb_err) #testing
  return FLD_loss_func



In [14]:
FLD_model, model_Ol = FLD_model_build()
sgd = optimizers.SGD(lr=0.0001, decay=0, momentum=momentum, nesterov=True, clipnorm=0.1)
loss1 = loss_function(model_Ol)
FLD_model.compile(optimizer=sgd,
              loss=[loss1],
              metrics=['accuracy'])


Input model Tensor("separable_conv2d_1/BiasAdd:0", shape=(?, 128, 96, 96), dtype=float32)
Local subnet Tensor("dropout_15/cond/Merge:0", shape=(?, 128, 96, 96), dtype=float32)
local_out Tensor("leaky_re_lu_16/LeakyRelu:0", shape=(?, 68, 96, 96), dtype=float32)
kernel Tensor("lambda_1/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Main 1x1 Tensor("lambda_1/depthwise:0", shape=(?, 68, 96, 96), dtype=float32)
local_out Tensor("conv2d_2/BiasAdd:0", shape=(?, 68, 96, 96), dtype=float32)
kernel Tensor("Ol/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Local output Tensor("Ol/depthwise:0", shape=(?, 68, 96, 96), dtype=float32)
Concatenate Tensor("concatenate_1/concat:0", shape=(?, 136, 96, 96), dtype=float32)
local_out Tensor("conv2d_3_target:0", shape=(?, ?, ?, ?), dtype=float32)
kernel Tensor("loss/conv2d_3_loss/Const:0", shape=(45, 45, 68, 1), dtype=float32)
Win ()
Loss Tensor("loss/conv2d_3_loss/Mul_2:0", shape=(), dtype=float32)


In [15]:
 FLD_model.fit(train_x, train_y, batch_size=8, epochs=10, validation_data=(test_x, test_y))

Train on 500 samples, validate on 92 samples
Epoch 1/10
500/500 [==============================] - 189s 379ms/step - loss: 65278529.7280 - acc: 0.0146 - val_loss: 4300995.9620 - val_acc: 0.0908
Epoch 2/10
500/500 [==============================] - 181s 362ms/step - loss: 63684381.7520 - acc: 0.0143 - val_loss: 4077344.6957 - val_acc: 0.0944
Epoch 3/10
500/500 [==============================] - 181s 362ms/step - loss: 62297526.6320 - acc: 0.0142 - val_loss: 3826862.9266 - val_acc: 0.0923
Epoch 4/10
500/500 [==============================] - 183s 367ms/step - loss: 60514290.1040 - acc: 0.0142 - val_loss: 3624207.2636 - val_acc: 0.0929
Epoch 5/10
500/500 [==============================] - 183s 366ms/step - loss: 59223950.8480 - acc: 0.0145 - val_loss: 3406733.1793 - val_acc: 0.0909
Epoch 6/10
500/500 [==============================] - 181s 362ms/step - loss: 57485226.0880 - acc: 0.0143 - val_loss: 3249088.2147 - val_acc: 0.0919
Epoch 7/10
500/500 [==============================] - 182s 36

In [0]:
# Prediction and visualization of landmarks on Image
img = []#get image
scale_x = image_W/img.shape[0]
scale_y = image_H/img.shape[1]

img1 = cv2.resize(img, (img_rows, img_cols))
img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img1 = cv2.normalize(img1,None,0,1,cv2.NORM_MINMAX)
pred = FLD_model.predict(img1)
landmarks = []
for p in pred:
  landmark_x, landmark_y = p.argmax()#get the position
  landmark_x = landmark_x / scale_x
  landmark_y = landmark_y / scale_y
  landmarks.append((landmark_x, landmark_y))

  # Mark scaled landmarks in original image
  cv2.circle(img, (round(landmark_x), round(landmark_y)),  2, (255,255,255), -1)

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()